# SaturdAIs.BIO

## Data reading & Data preprocessing

In [39]:
import pandas as pd

df = pd.read_excel("./macros_AI_sat.xlsx", sheet_name="Hoja2")
df.rename(columns={"Tipo de Macro": "MacroType"}, inplace=True)
df = df.drop(["Unnamed: 4"],axis=1)
data_df = df[['Uniprot Code', 'MacroType', 'Secuencia', 'uniprot']]
data_df.head()

,Uniprot Code,MacroType,Secuencia,uniprot
0,O28751,AF-1521-like,MEVLFEAKVGDITLKLAQGDITQYPAKAIVNAANKRLEHGGGVAYA...,O28751
1,D3RWS7,AF-1521-like,MEVEVVRELEMDKLKVKLAGGDITKYPAEAIVNAANKYLEHGGGVA...,D3RWS7
2,D2RH24,AF-1521-like,MVVKKFGSVEVVLEKGDITKYPAEAIVNAANKYLEHGGGVALAIAK...,D2RH24
3,A0A0F7ICE9,AF-1521-like,MKPEVVLRFSGVEVRLVQGDITKYPAEAIVNAANRHLEHGGGVAYA...,A0A0F7ICE9
4,A0A075LQ95,AF-1521-like,MNLTELTFGNLTFKLAQGDITKLPAEAIVNAANKYLEHGGGVALAI...,A0A075LQ95


In [40]:
data_df['domain_0'] = df[["domain","accession","hit_start","hit_end"]].values.tolist()
for i in range(1,8):
    data_df['domain_'+str(i)] = df[["domain."+str(i),"accession."+str(i),"hit_start."+str(i),"hit_end."+str(i)]].values.tolist()

data = data_df[['Uniprot Code', 'MacroType', 'Secuencia', 'uniprot']]
data['domains'] = data_df[["domain_0","domain_1","domain_2","domain_3","domain_4","domain_5","domain_6","domain_7"]].values.tolist()
data["domains"] = data["domains"].apply(lambda x: [el for el in x if str(el[0]) != "nan"])

/Users/pj/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/pj/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/pj/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a cop

# DL Classification

In [41]:
import torch
from tqdm import tqdm_notebook as tqdm
from tape import TAPETokenizer, ProteinBertModel # ProteinLSTMModel
#NECESITA DE CUDA

In [42]:
import numpy as np 
embs = np.load("macros_bert.npy")
embs.shape

(165, 768)

## Plot Embeddings

In [297]:
import altair as alt

def plot_multidimensional(x, y, method):

    from sklearn.decomposition import PCA
    from sklearn.manifold import TSNE
    import umap
    
    if method=="pca": comps = PCA().fit_transform(x)
    elif method=="tsne": comps = TSNE().fit_transform(x)
    elif method=="umap": comps = umap.UMAP().fit_transform(x)
    
    df_dimRed = pd.DataFrame({'X1': comps[:, 0],  'X2': comps[:, 1], "y": y})
    selection = alt.selection_multi(fields=['y'])
    new_select = alt.selection_interval()
    
    color = alt.condition(selection | new_select, alt.Color('y:N',scale=alt.Scale(scheme='category20'), legend=None), alt.value('lightgray'))
    colorr = alt.condition(selection, alt.Color('y:N',scale=alt.Scale(scheme='category20'), legend=None), alt.value('lightgray'))
    

    scatter = alt.Chart(df_dimRed).add_selection(selection).add_selection(new_select).mark_circle(
        size=60
    ).encode(
        x="X1", y="X2", color=color, tooltip="y"
    ).properties(
        width=400, height=400
    )

    legend = alt.Chart(df_dimRed).add_selection(selection).add_selection(new_select).mark_rect().encode(
        y=alt.Y('y:N', axis=alt.Axis(orient='right'),title=None),
        color=color,tooltip="y"
    )
    
    bars = alt.Chart(df_dimRed).add_selection(selection).mark_bar().encode(
        y=alt.Y('y:N',title=None),
        x=alt.X('count():Q',title=None),
        color=colorr, tooltip="y"
    ).transform_filter(
        new_select
    )


    return alt.hconcat(scatter,legend) &  bars


In [298]:
plot_multidimensional(embs, data["MacroType"], method="pca")

alt.VConcatChart(...)

In [299]:
plot_multidimensional(embs, data["MacroType"], method="tsne")

alt.VConcatChart(...)

In [300]:
plot_multidimensional(embs, data["MacroType"], method="umap")

alt.VConcatChart(...)

In [3]:
macrotype = set(data["MacroType"].values.tolist())
data.head(1000)

,Uniprot Code,MacroType,Secuencia,uniprot,domains
0,O28751,AF-1521-like,MEVLFEAKVGDITLKLAQGDITQYPAKAIVNAANKRLEHGGGVAYA...,O28751,"[[MACRO, PS51154, 1, 192]]"
1,D3RWS7,AF-1521-like,MEVEVVRELEMDKLKVKLAGGDITKYPAEAIVNAANKYLEHGGGVA...,D3RWS7,"[[MACRO, PS51154, 3, 193]]"
2,D2RH24,AF-1521-like,MVVKKFGSVEVVLEKGDITKYPAEAIVNAANKYLEHGGGVALAIAK...,D2RH24,"[[MACRO, PS51154, 1, 191]]"
3,A0A0F7ICE9,AF-1521-like,MKPEVVLRFSGVEVRLVQGDITKYPAEAIVNAANRHLEHGGGVAYA...,A0A0F7ICE9,"[[MACRO, PS51154, 1, 194]]"
4,A0A075LQ95,AF-1521-like,MNLTELTFGNLTFKLAQGDITKLPAEAIVNAANKYLEHGGGVALAI...,A0A075LQ95,"[[MACRO, PS51154, 1, 190]]"
...,...,...,...,...,...
160,P0C6Y5,Virus-type,MSSKQFKILVNEDYQVNVPSLPIRDVLQEIKYCYRNGFEGYVFVPE...,P0C6Y5,"[[GLU_RICH, PS50313, 989, 1042], [PEPTIDASE_C1..."
161,A0A0P0LKV0,Virus-type,MACNRFTLAVASDSEISGTGCATVAQAVRLYSEAAVNGFRACRFVS...,A0A0P0LKV0,"[[RDRP_SSRNA_POS, PS50507, 4680, 4842], [PEPTI..."
162,A0A0U2GMU3,Virus-type,MACNRVTLAVASDTEISATGCSTIALAVRRYSEAASNGFRACRFVS...,A0A0U2GMU3,"[[RDRP_SSRNA_POS, PS50507, 4680, 4842], [PEPTI..."
163,B1PHJ4,Virus-type,MSSNLVTLAFASDSEISAEGFCDVSSAVYAFSVSAANGFTDCRFVA...,B1PHJ4,"[[PEPTIDASE_C16, PS51124, 1173, 1423], [PEPTID..."


In [4]:
domains_examples = data.domains.map(lambda x: [el[0] for el in x]).tolist()
domains_names = list(set([item for items in domains_examples for item in items]))
domains_names = sorted(domains_names)
domains_names

['ALPHAVIRUS_MT',
 'ASP_RICH',
 'BRCT',
 'CRAL_TRIO',
 'CV_ZBD',
 'CYS_RICH',
 'GLU_RICH',
 'HELICASE_ATP_BIND_1',
 'HELICASE_CTER',
 'LYS_RICH',
 'MACRO',
 'M_PRO',
 'NLS_BP',
 'NSP2PRO',
 'PARG_cat',
 'PARP_CATALYTIC',
 'PEPTIDASE_C16',
 'PSRV_HELICASE',
 'RDRP_SSRNA_POS',
 'SER_RICH',
 'SIRTUIN',
 'VAL_RICH',
 'WWE']

In [5]:
shape_list = ["<>","<>","()","^","v","o","v","^","<>","()","[]","v","^","()","<>","<>","()","^","v","o","v","^","<>"]
color_list = ["orange","yellow","pink","purple","blue","green","gold","cyan","greenyellow","sienna","red","lightsalmon","teal","magenta","lightblue","firebrick","aquamarine","darkcyan","olive","seagreen","limegreen","indigo","chocolate"]
color_dict = dict(zip(domains_names, color_list))
shape_dict = dict(zip(domains_names, shape_list))

In [6]:
import sys
from ete3 import Tree, SeqMotifFace, TreeStyle, add_face_to_node, COLOR_SCHEMES,SVG_COLORS
#SVG_COLORS
#COLOR_SCHEMES.keys()

In [ ]:
import sys
from ete3 import Tree, SeqMotifFace, TreeStyle, add_face_to_node, COLOR_SCHEMES

seq = "D"
motifs = []
# seq.start, seq.end, shape, width, height, fgcolor, bgcolor
# [10, 100, "[]", None, 10, "black", "rgradient:blue", "arial|8|white|MACRO"],

def layout(node):
    if node.is_leaf():
        seqFace = SeqMotifFace(seq, motifs, scale_factor=1,gap_format="line",gapcolor='black')
        add_face_to_node(seqFace, node, 0, position="aligned")
       

def get_example_tree(leafs,names):
    # Create a random tree and add to each leaf a random set of motifs from the original set
    t = Tree()       #CREATE TREE
    t.populate(leafs,names)   #NUMBER OF TREE LEAFS WITH MOTIFS
    
    for i,l in enumerate(t.iter_leaves()):
        print(i,"->>",l)
        seq = data.loc[i,"Secuencia"]
        seq_code = data.loc[i,"Uniprot Code"]
        print(seq[0:30])
        dominios_seq = data.loc[i,"domains"]
        print(dominios_seq,"\n")
        motifs = [[int(dominio[2]),int(dominio[3]),shape_dict[dominio[0]],int(dominio[3])-int(dominio[2]),10,color_dict[dominio[0]],"rgradient:"+color_dict[dominio[0]],"arial|8|black|"+dominio[0]] for dominio in dominios_seq]
        print(motifs,"\n")
        seq_motifs = [list(m) for m in motifs]
        print(seq_motifs,"\n\n\n") #puedo dar seq aquí, pero diría que no es lo correcto
        seqFace = SeqMotifFace(seq=None, motifs=seq_motifs, scale_factor=1,gap_format="line",gapcolor='black')
        seqFace.margin_bottom = 4
        f = l.add_face(seqFace, 0, "aligned")
        #f = l.add_face(textFace(seq_code), column=i, position="")

    ts = TreeStyle()
    ts.layout_fn = layout
    return t, ts

if __name__ == '__main__':
    t, ts = get_example_tree(len(data),data["Uniprot Code"].values.tolist())
    t.render("tree2.png",tree_style=ts)
    #t.render("%%inline",w=1200, dpi=300,tree_style=ts)
    t.show(tree_style=ts)
    print("\n\nDone.")

0 ->> 
--P0C6U6
MEVLFEAKVGDITLKLAQGDITQYPAKAIV
[['MACRO', 'PS51154', 1, 192]] 

[[1, 192, '[]', 191, 10, 'red', 'rgradient:red', 'arial|8|black|MACRO']] 

[[1, 192, '[]', 191, 10, 'red', 'rgradient:red', 'arial|8|black|MACRO']] 



1 ->> 
--U4TVH3
MEVEVVRELEMDKLKVKLAGGDITKYPAEA
[['MACRO', 'PS51154', 3, 193]] 

[[3, 193, '[]', 190, 10, 'red', 'rgradient:red', 'arial|8|black|MACRO']] 

[[3, 193, '[]', 190, 10, 'red', 'rgradient:red', 'arial|8|black|MACRO']] 



2 ->> 
--A0A0A2UAG7
MVVKKFGSVEVVLEKGDITKYPAEAIVNAA
[['MACRO', 'PS51154', 1, 191]] 

[[1, 191, '[]', 190, 10, 'red', 'rgradient:red', 'arial|8|black|MACRO']] 

[[1, 191, '[]', 190, 10, 'red', 'rgradient:red', 'arial|8|black|MACRO']] 



3 ->> 
--R9KG45
MKPEVVLRFSGVEVRLVQGDITKYPAEAIV
[['MACRO', 'PS51154', 1, 194]] 

[[1, 194, '[]', 193, 10, 'red', 'rgradient:red', 'arial|8|black|MACRO']] 

[[1, 194, '[]', 193, 10, 'red', 'rgradient:red', 'arial|8|black|MACRO']] 



4 ->> 
--WP_027129123.1
MNLTELTFGNLTFKLAQGDITKLPAEAIVN
[['MACRO', 'P

[[1021, 1275, '()', 254, 10, 'aquamarine', 'rgradient:aquamarine', 'arial|8|black|PEPTIDASE_C16'], [1640, 1886, '()', 246, 10, 'aquamarine', 'rgradient:aquamarine', 'arial|8|black|PEPTIDASE_C16'], [1253, 1421, '[]', 168, 10, 'red', 'rgradient:red', 'arial|8|black|MACRO'], [2940, 3242, 'v', 302, 10, 'lightsalmon', 'rgradient:lightsalmon', 'arial|8|black|M_PRO']] 

[[1021, 1275, '()', 254, 10, 'aquamarine', 'rgradient:aquamarine', 'arial|8|black|PEPTIDASE_C16'], [1640, 1886, '()', 246, 10, 'aquamarine', 'rgradient:aquamarine', 'arial|8|black|PEPTIDASE_C16'], [1253, 1421, '[]', 168, 10, 'red', 'rgradient:red', 'arial|8|black|MACRO'], [2940, 3242, 'v', 302, 10, 'lightsalmon', 'rgradient:lightsalmon', 'arial|8|black|M_PRO']] 



155 ->> 
--A0A0U1WHD1
MEKPVVNVDVDPQSPFVVQLQKSFPQFEVV
[['SER_RICH', 'PS50324', 1718, 1746], ['RDRP_SSRNA_POS', 'PS50507', 2267.0, 2382.0], ['MACRO', 'PS51154', 1338.0, 1507.0], ['NSP2PRO', 'PS51520', 1012.0, 1341.0], ['PSRV_HELICASE', 'PS51657', 695.0, 999.0], ['ALPH

In [ ]:
#http://etetoolkit.org/docs/2.3/tutorial/tutorial_drawing.html#phylogenetic-trees-and-sequence-domains
#http://etetoolkit.org/ipython_notebook/
#https://www.youtube.com/watch?v=7yfMwmaptf0
#http://evomicsorg.wpengine.netdna-cdn.com/wp-content/uploads/2019/01/ete_tutorial.pdf
#http://etetoolkit.org/docs/latest/reference/reference_treeview.html
